In [9]:
import fnmatch, re, os
import json

def parametrizedFileSelection(fileDirectory, fileArguments):
    files = []
    for file in os.listdir(fileDirectory):
        for key in fileArguments:
            match = False
            for value in fileArguments[key]:
                expression = key + "=" + str(value)
                if expression in file:
                    match = True
            if match == False:
                break
    
        if match == True:
            files.append(file)
        
    return files


In [15]:
fileDirectory = "../outputs/facebook_combined_vobws/"
files = parametrizedFileSelection(fileDirectory, { "nWorkers": [10]})

print(files)
for file in files:
    with open(fileDirectory + file) as json_file:
        data = json.load(json_file)
        print(data['workers'])
            

['filename=facebook_combined,nWorkers=10,voterSeed=0,comDet=community_multilevel,assignment=generateArrangementNumber,withBoundary=False,distType=close,minDist=1_0.json', 'filename=facebook_combined,nWorkers=10,voterSeed=0,comDet=community_multilevel,assignment=generateArrangementNumber,withBoundary=False,distType=close,minDist=2_100.json', 'filename=facebook_combined,nWorkers=10,voterSeed=0,comDet=community_multilevel,assignment=generateArrangementNumber,withBoundary=False,distType=close,minDist=3_200.json', 'filename=facebook_combined,nWorkers=10,voterSeed=0,comDet=community_multilevel,assignment=generateArrangementNumber,withBoundary=False,distType=close,minDist=4_300.json', 'filename=facebook_combined,nWorkers=10,voterSeed=0,comDet=community_multilevel,assignment=generateArrangementNumber,withBoundary=False,distType=far,minDist=4_400.json', 'filename=facebook_combined,nWorkers=10,voterSeed=0,comDet=community_multilevel,assignment=generateArrangementNumber,withBoundary=False,distTyp

['V1286', 'V3844', 'V3390', 'V1338', 'V1516', 'V2986', 'V3459', 'V2692', 'V3883', 'V3952']
['V1286', 'V3844', 'V3390', 'V1338', 'V1516', 'V2986', 'V3459', 'V2692', 'V3883', 'V3952']
['V1286', 'V3844', 'V3390', 'V1338', 'V1516', 'V2986', 'V3459', 'V2692', 'V3883', 'V3952']
['V1713', 'V448', 'V1419', 'V446', 'V2509', 'V3971', 'V863', 'V3147', 'V3350', 'V3520']
['V1713', 'V448', 'V1419', 'V446', 'V2509', 'V3971', 'V863', 'V3147', 'V3350', 'V3520']
['V1713', 'V448', 'V1419', 'V446', 'V2509', 'V3971', 'V863', 'V3147', 'V3350', 'V3520']
['V1713', 'V448', 'V1419', 'V446', 'V2509', 'V3971', 'V863', 'V3147', 'V3350', 'V3520']
['V1713', 'V448', 'V1419', 'V446', 'V2509', 'V3971', 'V863', 'V3147', 'V3350', 'V3520']
['V1713', 'V448', 'V1419', 'V446', 'V2509', 'V3971', 'V863', 'V3147', 'V3350', 'V3520']
['V1713', 'V448', 'V1419', 'V446', 'V2509', 'V3971', 'V863', 'V3147', 'V3350', 'V3520']
['V1713', 'V448', 'V1419', 'V446', 'V2509', 'V3971', 'V863', 'V3147', 'V3350', 'V3520']
['V1713', 'V448', 'V141